In [1]:
# # Step 1: Clean installation
# !pip uninstall surya-ocr transformers torch torchvision -y

# # Step 2: Install specific compatible versions
# !pip install torch==2.1.2 torchvision==0.16.2 --index-url https://download.pytorch.org/whl/cu118
# !pip install transformers==4.37.2
# !pip install surya-ocr==0.4.15

# # Step 3: Install additional dependencies
# !pip install pdf2image pillow pandas requests

# # Step 4: RESTART YOUR KERNEL (this is crucial!)
# import os, sys; os._exit(0)  # Restart after install


In [ ]:
# !pip install torch==2.1.2 torchvision==0.16.2 --index-url https://download.pytorch.org/whl/cu118
# !pip install transformers==4.37.2
# !pip install surya-ocr==0.4.15

# import importlib
# import surya
# importlib.reload(surya)


In [2]:
# initialize empty repo
!git init /kaggle/working/lk_legal_docs
%cd /kaggle/working/lk_legal_docs

# add remote and fetch only the branch
!git remote add origin https://github.com/nuuuwan/lk_legal_docs.git
!git fetch --depth 1 origin data

# enable sparse checkout
!git sparse-checkout init --cone
!git sparse-checkout set data/lk_acts

# pull only the lk_acts folder
!git pull origin data


hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /kaggle/working/lk_legal_docs/.git/
/kaggle/working/lk_legal_docs
remote: Enumerating objects: 270809, done.
remote: Counting objects: 100% (270809/270809), done.
remote: Compressing objects: 100% (94466/94466), done.
Receiving objects: 100% (270809/270809), 7.04 GiB | 33.34 MiB/s, done.
remote: Total 270809 (delta 86210), reused 270381 (delta 86007), pack-reused 0 (from 0)
Resolving deltas: 100% (86210/86210), done.
From https://github.com/nuuuwan/lk_legal_docs
 * branch   

In [3]:
import pandas as pd
import numpy as np

In [4]:
!apt-get update -y
!apt-get install -y poppler-utils

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:4 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]      
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease    
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Get:9 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [80.4 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,008 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]          
Get:12 http://archive.ubuntu.com/ubuntu jammy-

In [5]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 3.0 MB/s eta 0:00:000:00:0100:01


In [ ]:
import os, json, requests, pandas as pd
from pdf2image import convert_from_bytes
from PIL import Image
import pytesseract
from tqdm import tqdm
import torch

# surya imports
from surya.model.detection.model import load_model as load_det_model
from surya.model.recognition.model import load_model as load_rec_model
from surya.model.recognition.processor import load_processor as load_rec_processor
from surya.ocr import run_ocr
from transformers import SegformerImageProcessor

DATA_DIR = "/kaggle/working/lk_legal_docs/data/lk_acts"
SAVE_PATH = "/kaggle/working/acts_si.csv"

# -------------------------------
# initialize surya models (cpu)
# -------------------------------
print("Initializing Surya OCR models (CPU)...")
device = torch.device("cpu")
det_model = load_det_model().to(device)
rec_model = load_rec_model().to(device)
rec_processor = load_rec_processor()
try:
    det_processor = SegformerImageProcessor.from_pretrained("vikp/surya_det3")
except Exception:
    det_processor = None
print("✅ Surya models ready on CPU!")

# -------------------------------
# helper functions
# -------------------------------
def download_pdf(url, timeout=30):
    try:
        resp = requests.get(url, timeout=timeout, headers={"User-Agent": "Mozilla/5.0"})
        if resp.status_code == 200:
            return resp.content
    except requests.RequestException:
        pass
    return None


def is_unicode_sinhala(text):
    """Check if text contains Sinhala unicode characters"""
    return any("\u0d80" <= ch <= "\u0dff" for ch in text)


def ocr_page(img, langs=["si"]):
    """
    OCR a single page:
    - first try pytesseract (extractable text)
    - if not usable, fallback to Surya OCR
    """
    # try pytesseract first
    try:
        text = pytesseract.image_to_string(img, lang="si")
        if is_unicode_sinhala(text) and len(text.strip()) > 5:
            return text
    except Exception:
        pass

    # fallback to Surya OCR
    preds = run_ocr([img], [langs], det_model, det_processor, rec_model, rec_processor)
    if preds and preds[0].text_lines:
        page_text = " ".join([line.text for line in preds[0].text_lines])
        return page_text
    return ""



def ocr_pdf(pdf_bytes, langs=["si"]):
    """OCR pdf page by page with tqdm progress bar"""
    images = convert_from_bytes(pdf_bytes, dpi=200)
    full_text = ""

    for i, img in enumerate(tqdm(images, desc="Processing pages", unit="page")):
        page_text = ocr_page(img, langs=langs)
        full_text += page_text + "\n\n"


    return full_text.strip()


def save_results(results, save_path):
    if not results:
        return
    df = pd.DataFrame(results)
    if not os.path.exists(save_path):
        df.to_csv(save_path, index=False, encoding="utf-8-sig")
    else:
        df.to_csv(save_path, index=False, mode="a", header=False, encoding="utf-8-sig")


# -------------------------------
# process PDFs from doc.json
# -------------------------------
def process_docjson(root_dir, save_path):
    processed = set()
    if os.path.exists(save_path):
        try:
            existing_df = pd.read_csv(save_path)
            processed.update(existing_df["name"].astype(str).tolist())
        except Exception:
            pass

    results = []
    for subdir, _, files in os.walk(root_dir):
        if "doc.json" in files:
            json_path = os.path.join(subdir, "doc.json")
            try:
                with open(json_path, "r", encoding="utf-8") as f:
                    doc = json.load(f)

                if doc.get("lang") != "si":
                    continue
                pdf_url = doc.get("url_pdf")
                if not pdf_url:
                    continue

                pdf_name = os.path.basename(pdf_url)
                if pdf_name in processed:
                    continue

                print(f"Processing {pdf_name} ...", flush=True)
                pdf_bytes = download_pdf(pdf_url)
                if pdf_bytes:
                    text = ocr_pdf(pdf_bytes, langs=["si"])
                    print(f"{pdf_name} extracted, total chars: {text[:500]}\n", flush=True)

                    results.append({
                        "name": pdf_name,
                        "url": pdf_url,
                        "text": text,
                        "character_count": len(text),
                        "status": "success" if text else "no_text"
                    })
                else:
                    results.append({
                        "name": pdf_name,
                        "url": pdf_url,
                        "text": "",
                        "character_count": 0,
                        "status": "download_failed"
                    })

                if len(results) >= 2:  # save small batches
                    save_results(results, save_path)
                    results = []
            except Exception as e:
                print(f"Error in {json_path}: {e}", flush=True)
                results.append({
                    "name": f"ERROR_{os.path.basename(json_path)}",
                    "url": "",
                    "text": "",
                    "character_count": 0,
                    "status": f"error: {str(e)[:100]}"
                })

    if results:
        save_results(results, save_path)
    print("✅ Finished! Results saved to", save_path, flush=True)


# -------------------------------
# main
# -------------------------------
if __name__ == "__main__":
    if not os.path.exists(DATA_DIR):
        print(f"❌ Data directory does not exist: {DATA_DIR}", flush=True)
    else:
        process_docjson(DATA_DIR, SAVE_PATH)


Initializing Surya OCR models (CPU)...


The following layers were not sharded: decode_head.linear_c.*.proj.bias, vit.stages.*.blocks.*.main.spatial_conv.norm.running_var, vit.stages.*.blocks.*.context_module.main.proj.norm.num_batches_tracked, vit.stages.*.blocks.*.main.point_conv.norm.running_mean, vit.stages.*.blocks.*.context_module.main.proj.norm.bias, vit.stem.res*.main.conv*.conv.weight, vit.stages.*.blocks.*.main.spatial_conv.norm.running_mean, vit.stages.*.blocks.*.local_module.main.inverted_conv.conv.weight, vit.stem.res*.main.conv*.norm.running_mean, decode_head.linear_c.*.proj.weight, vit.stages.*.blocks.*.local_module.main.point_conv.norm.running_var, vit.stem.res*.main.conv*.norm.bias, decode_head.batch_norm.running_var, vit.stages.*.blocks.*.local_module.main.depth_conv.conv.weight, decode_head.classifier.bias, decode_head.batch_norm.bias, vit.stages.*.blocks.*.main.inverted_conv.conv.bias, vit.stages.*.blocks.*.local_module.main.point_conv.norm.running_mean, vit.stages.*.blocks.*.local_module.main.point_conv.c

Loaded detection model vikp/surya_det3 on device cpu with dtype torch.float32
Loaded recognition model vikp/surya_rec on device cpu with dtype torch.float32
✅ Surya models ready on CPU!
Processing 05-2024_S.pdf ...


/usr/local/lib/python3.11/dist-packages/transformers/image_processing_base.py:417: UserWarning: The following named arguments are not valid for `SegformerImageProcessor.__init__` and were ignored: '_valid_processor_keys'
  image_processor = cls(**image_processor_dict)
Detecting bboxes: 100%|██████████| 1/1 [00:09<00:00,  9.70s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:09<00:00,  9.66s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:09<00:00,  9.05s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:09<00:00,  9.28s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:09<00:00,  9.10s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:09<00:00,  9.21s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:09<00:00,  9.10s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:09<00:00,  9.10s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:09<00:00,  9.44s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:09<00:00,  9.12s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:09<00:00,  9.35s/it]

Detecting bboxes

05-2024_S.pdf extracted, total chars: ශ්‍රී ලංකා ප්‍රජාතාන්ත්‍රික සමාජවාදී ජනරජයේ පාර්ලිමේන්තුව 2024 අංක 5 දරන සමථකරණය තුළින් ඇතිවන්නා වූ අන්තර්ජාතික නිරවුල්කරණ ගිවිසුම් පිළිගැනීමේ සහ බලාත්මක කිරීමේ පනත [සහතිකය සටහන් කළේ 2024 ජනවාරි මස 31 වන දින] ආණ්ඩුවේ නියමය පරිදි මුද්‍රණය කරන ලදී 2024 පෙබරවාරි මස 02 වන දින ශ්‍රී ලංකා ප්‍රජාතාන්ත්‍රික සමාජවාදී ජනරජයේ ගැසට් පත්‍රයේ II වන කොටසේ අතිරේකයක් වශයෙන් පළ කරන ලදී  ශ්‍රී ලංකා රජයේ මුද්‍රණ දෙපාර්තමේන්තුවේ මුද්‍රණය කරන ලදී.  කොළඹ 5, රජයේ ප්‍රකාශන කාර්යාංශයෙන් මිලදී ලබාගත හැකි ය. මිල : රු. 24.0

Processing 02-2024_S.pdf ...



Detecting bboxes: 100%|██████████| 1/1 [00:09<00:00,  9.06s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:09<00:00,  9.12s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:09<00:00,  9.07s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:09<00:00,  9.26s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:09<00:00,  9.34s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:09<00:00,  9.16s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:09<00:00,  9.17s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:09<00:00,  9.60s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:09<00:00,  9.90s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:09<00:00,  9.65s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:10<00:00, 10.21s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:10<00:00, 10.11s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:10<00:00, 10.54s/it]

Processing pages: 100%|██████████| 13/13 [16:24<00:00, 75.73s/page]

02-2024_S.pdf extracted, total chars: ශ්‍රී ලංකා ප්‍රජාතාන්ත්‍රික සමාජවාදී ජනරජයේ පාර්ලිමේන්තුව 2024 අංක 2 දරන සමථ මණ්ඩල (සංශෝධන) පනත [සහතිකය සටහන් කළේ 2024 ජනවාරි මස 23 වන දින] ආණ්ඩුවේ නියමය පරිදි මුද්‍රණය කරන ලදී 2024 ජනවාරි මස 26 වන දින ශ්‍රී ලංකා ප්‍රජාතාන්ත්‍රික සමාජවාදී ජනරජයේ ගැසට් පත්‍රයේ II වන කොටසේ අතිරේකයක් වශයෙන් පළ කරන ලදී  ශ්‍රී ලංකා රජයේ මුද්‍රණ දෙපාර්තමේන්තුවේ මුද්‍රණය කරන ලදී.  කොළඹ 5, රජයේ ප්‍රකාශන කාර්යාංශයෙන් මිලදී ලබාගත හැකි ය. මිල : රු. 24.00 තැපැල් ගාස්තුව : රු. 150.00  මෙම පනත www.documents.gov.lk වෙබ් අඩවියෙ

Processing 17-2024_S.pdf ...



Detecting bboxes: 100%|██████████| 1/1 [00:12<00:00, 12.61s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:11<00:00, 11.20s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:10<00:00, 10.59s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:10<00:00, 10.79s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:10<00:00, 10.72s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:10<00:00, 10.29s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:10<00:00, 10.91s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:11<00:00, 11.10s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:11<00:00, 11.41s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:11<00:00, 11.49s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:28<00:00, 28.85s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:10<00:00, 10.91s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:10<00:00, 10.68s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:11<00:00, 11.42s/it]

Detecting bboxes: 100%|██████████| 1/1 [00:11<00:00, 11.41s/it]

Detecting bboxes: 100%|█